# Problem 1
## (a)

We use EM algorithm to estimate $\pi$ and $\sigma^2$, and calculate $\hat \mu_i^S$ by the algorithm's output.

From the hierarchical model，we can derive
$$
z_i \mid {\gamma_i = 0} \sim \mathcal{N}(0, 1), \\
z_i \mid {\gamma_i = 1 }\sim \mathcal{N}(0, 1 + \sigma^2),
$$

For each i
$$
p(z_i,\gamma_i|\pi,\sigma^2) = p(\gamma_i|\pi)p(z_i|\gamma_i,\sigma^2)
$$

By the defination

$$
p(\gamma_i|\pi) = \pi^{\gamma_i}(1-\pi)^{1-\gamma_i}
\\
p(z_i|\gamma_i,\sigma^2)=[\phi(z_i;0,1+\sigma^2)]^{\gamma_i}[\phi(z_i;0,1)]^{(1-\gamma_i)}
$$

where 
$
\phi(z;a,b) = (2\pi b)^{-1/2} \exp\!\left(-\frac{(z-a)^2}{2b}\right).
$ is the probabilty density funtion of normal distribution

Thus we can deduce
$$
p(z_i,\gamma_i|\pi,\sigma^2)=[\pi\phi(z_i;0,1+\sigma^2)]^{\gamma_i}[\pi\phi(z_i;0,1)]^{(1-\gamma_i)}
$$

And then the complete-data log-likelihood is

$$
\ell_c(\pi,\sigma^2)
=\sum_{i=1}^N
\Big[
\gamma_i\big(\log\pi+\log\phi(z_i;0,1+\sigma^2)\big)
+(1-\gamma_i)\big(\log(1-\pi)+\log\phi(z_i;0,1)\big)
\Big],
$$



which equal to 

$$
\ell_c(\pi,\sigma^2)
=\sum_{i=1}^N
\Big[
\gamma_i\Big(\log\pi - \frac{1}{2}\log(1+\sigma^2) - \frac{z_i^2}{2(1+\sigma^2)}\Big)
+(1-\gamma_i)\log(1-\pi)
\Big] + C
$$





$C$ includes all terms that are unrelated to $\pi$ and $\sigma^2$

### E step

Define the responsibility
$$
w_i^{(t)} = \mathbb E[\gamma_i \mid z_i;\pi^{(t)},\sigma^{2(t)}]
= P(\gamma_i=1\mid z_i;\pi^{(t)},\sigma^{2(t)}).
= \frac{\pi^{(t)}p(z_i|\gamma_i=1,\sigma^{2(t)})}{\pi^{(t)}p(z_i|\gamma_i=1,\sigma^{2(t)})+(1-\pi^{(t)})p(z_i|\gamma_i=0)}.
$$


Thus 
$$
Q(\pi,\sigma^2|\pi^{(t)},\sigma^{2(t)}) = \sum_{i=1}^N [w_i^{(t)}(\log\pi - \frac{1}{2}\log(1+\sigma^2) - \frac{z_i^2}{2(1+\sigma^2)} )+(1-w_i^{(t)})\log(1-\pi)] +C
$$

### M step


$$
\frac{\partial Q}{\partial \pi}
=\sum_{i=1}^N\Big(\frac{w_i^{(t)}}{\pi}-\frac{1-w_i^{(t)}}{1-\pi}\Big)=0
\quad\Rightarrow\quad
\pi^{(t+1)}=\frac{1}{N}\sum_{i=1}^N w_i^{(t)}.
$$




$$
\frac{\partial Q}{\partial \sigma^2}
=\sum_{i=1}^N w_i^{(t)}\Big(-\frac{1}{2(1+\sigma^2)}+\frac{z_i^2}{2(1+\sigma^2)^2}\Big)=0.
\quad\Rightarrow\quad
\sigma^{2(t+1)}=\frac{\sum_{i=1}^N w_i^{(t)} z_i^2}{\sum_{i=1}^N w_i^{(t)}}-1. (\ge 0)

$$



Repeat E step and M step until the parameters converge, yielding $\hat \pi$ and $\hat \sigma^2$, using these estimates, compute the posterior mean

$$
\hat{w}_{i}=\frac{\hat{\pi}\mathrm{~}\phi{\left(z_{i};0,\mathrm{~}1+\hat{\sigma}^{2}\right)}}{\hat{\pi}\mathrm{~}\phi{\left(z_{i};0,\mathrm{~}1+\hat{\sigma}^{2}\right)}+(1-\hat{\pi})\phi{\left(z_{i};0,\mathrm{~}1\right)}}
\\
\hat{\mu}_{i}^{S}=\hat{w}_{i}\frac{\hat{\sigma}^{2}}{1+\hat{\sigma}^{2}}z_{i}.
$$

## (b)

### Function

For Tweedie’s formula, we estimate the marginal density $f(z)$ using KDE with the Gaussian kernel $\phi(u)=\frac{1}{\sqrt{2\pi}}e^{-u^{2}/2}$:


$$
\hat{f}(z)=\frac{1}{Nh}\sum_{j=1}^N\phi{\left(\frac{z-z_j}{h}\right)}.
$$

The bandwidth $h$ is selected using Silverman’s rule:
$h = 1.06\,\hat{s}\,N^{-1/5}$, where $\hat{s}$ is the sample standard deviation.

Thus the score function is 

$$
\frac{d}{dz}\log\hat{f}(z)=\frac{\sum_{j=1}^N\frac{z-z_j}{h^2}\phi{\left(\frac{z-z_j}{h}\right)}}{\sum_{j=1}^N\phi{\left(\frac{z-z_j}{h}\right)}}
$$

thus the Tweedie estimator is

$$
\hat{\mu}_i^{\mathrm{Tw}}=z_i+\frac{d}{dz}\log\hat{f}(z_i) = z_i +\frac{\sum_{j=1}^N\frac{z_i-z_j}{h^2}\phi{\left(\frac{z_i-z_j}{h}\right)}}{\sum_{j=1}^N\phi{\left(\frac{z_i-z_j}{h}\right)}}
$$


In [ ]:
##
# Tweedie's formular 
# KDE to estimate the density function
##
import numpy as np
def tweedie(z, tau2=1.0, eps=1e-12):
    z = np.asarray(z, dtype=float).ravel()
    N = z.size

    if N == 0:
        return np.array([]), np.nan
    if N == 1:
        return z.copy(), 1.0

    # --- Silverman's rule of thumb (classic) ---
    s = np.std(z, ddof=1)  # unbiased std
    if s == 0:
        s = 1.0  # fallback if all z are identical
    h = 1.06 * s * (N ** (-1.0 / 5.0))

    # Precompute constants
    inv_h = 1.0 / h
    inv_h2 = inv_h * inv_h

    # Pairwise differences: shape (N, N)
    diff = z[:, None] - z[None, :]          # z_i - z_j
    u = diff * inv_h                        # (z_i - z_j) / h

    # Gaussian kernel (constant factor cancels, so omit 1/sqrt(2π))
    K = np.exp(-0.5 * u * u)                # proportional to φ(u)

    # Denominator: sum_j K_ij
    den = np.sum(K, axis=1)

    # Numerator: sum_j (z_i - z_j)/h^2 * K_ij
    num = np.sum(diff * K, axis=1) * inv_h2

    # Score = d/dz log f_hat(z_i)
    score = num / np.maximum(den, eps)

    # Tweedie estimate
    mu_hat = z + tau2 * score

    return mu_hat


JSE:  
$$
\hat{\boldsymbol{\mu}}^{\mathrm{~JS}}=\left(1-\frac{d-2}{\|\mathbf{z}\|^{2}}\right)_{+}\mathbf{z},
$$

In [ ]:

def JSE(z, sigma2=1.0, center=None, positive_part=True):

    z = np.asarray(z, dtype=float)
    original_shape = z.shape

    if z.ndim == 1:
        z = z[None, :]  # shape (1, d)
        squeeze_output = True
    else:
        squeeze_output = False

    N, d = z.shape

    if d < 3:
        raise ValueError("James-Stein estimator is only defined for dimension d >= 3")
        pass

    # Set center
    if center is None:
        center = np.zeros(d)
    else:
        center = np.asarray(center, dtype=float)
        if center.shape != (d,):
            raise ValueError(f"center must be shape ({d},), got {center.shape}")

    # Center the data
    z_centered = z - center  # shape (N, d)

    # Squared norm: ||z - center||^2, shape (N,)
    norm_sq = np.sum(z_centered**2, axis=1)

    # Avoid division by zero
    eps = np.finfo(float).eps
    norm_sq = np.maximum(norm_sq, eps)

    # Shrinkage factor: (d - 2) * sigma2 / ||z - center||^2
    shrinkage = (d - 2) * sigma2 / norm_sq  # shape (N,)

    if positive_part:
        factor = np.maximum(1.0 - shrinkage, 0.0)
    else:
        factor = 1.0 - shrinkage

    # Apply shrinkage
    mu_hat = center + (factor[:, None] * z_centered)

    # Reshape to match input
    if squeeze_output:
        mu_hat = mu_hat[0]

    return mu_hat

EM：Provided in （a）

In [ ]:

def em_spike_slab(z, pi_init=0.2, sigma2_init=1.0, tol=1e-6, max_iter=1000,
                  min_sigma2=1e-8, return_history=False):

    z = np.asarray(z, dtype=float).ravel()
    N = z.size

    # --- helpers ---
    def log_norm_pdf(x, mean, var):
        return -0.5 * (np.log(2.0 * np.pi * var) + (x - mean) ** 2 / var)

    def obs_loglik(pi, sigma2):
        # log [ pi * N(0,1+sigma2) + (1-pi) * N(0,1) ] summed over i
        la = np.log(pi)   + log_norm_pdf(z, 0.0, 1.0 + sigma2)
        lb = np.log(1-pi) + log_norm_pdf(z, 0.0, 1.0)
        m = np.maximum(la, lb)
        return np.sum(m + np.log(np.exp(la - m) + np.exp(lb - m)))

    # --- init ---
    pi = float(np.clip(pi_init, 1e-6, 1-1e-6))
    sigma2 = float(max(sigma2_init, min_sigma2))
    last_ll = -np.inf

    if return_history:
        hist_pi, hist_s2, hist_ll = [], [], []

    for t in range(1, max_iter + 1):
        # E-step: responsibilities w_i = P(gamma=1 | z_i, pi, sigma2)
        la = np.log(pi)   + log_norm_pdf(z, 0.0, 1.0 + sigma2)
        lb = np.log(1-pi) + log_norm_pdf(z, 0.0, 1.0)
        m  = np.maximum(la, lb)
        a  = np.exp(la - m)
        b  = np.exp(lb - m)
        w  = a / (a + b)                 # shape (N,)

        # M-step:
        pi_new = np.clip(np.mean(w), 1e-12, 1 - 1e-12)
        denom = np.sum(w)
        if denom < 1e-16:
            sigma2_new = min_sigma2      # degenerate case: all weights ~ 0
        else:
            sigma2_new = np.sum(w * z**2) / denom - 1.0
            sigma2_new = float(max(sigma2_new, min_sigma2))

        # Check convergence via observed-data log-likelihood
        ll = obs_loglik(pi_new, sigma2_new)

        if return_history:
            hist_pi.append(pi_new)
            hist_s2.append(sigma2_new)
            hist_ll.append(ll)

        if np.isfinite(last_ll) and abs(ll - last_ll) <= tol * (1.0 + abs(last_ll)):
            pi, sigma2 = pi_new, sigma2_new
            break

        pi, sigma2 = pi_new, sigma2_new
        last_ll = ll

    # Posterior mean with the learned params
    la = np.log(pi)   + log_norm_pdf(z, 0.0, 1.0 + sigma2)
    lb = np.log(1-pi) + log_norm_pdf(z, 0.0, 1.0)
    m  = np.maximum(la, lb)
    a  = np.exp(la - m)
    b  = np.exp(lb - m)
    w  = a / (a + b)

    shrink = sigma2 / (1.0 + sigma2)
    mu_hat = w * shrink * z

    if return_history:
        return pi, sigma2, mu_hat, {
            "pi": np.array(hist_pi),
            "sigma2": np.array(hist_s2),
            "loglik": np.array(hist_ll),
            "iters": t,
        }
    return pi, sigma2, mu_hat



pi_hat=0.0773, sigma2_hat=2.6184, iterations=73
MSE(EM posterior mean) = 0.1652


### Simulation

Generate data for the normal means model with a spike–slab prior:

$$
gamma_i ~ Bernoulli(pi)
mu_i | gamma_i = 1 ~ N(0, sigma2),  mu_i | gamma_i = 0 = 0
z_i = mu_i + eps_i,  eps_i ~ N(0, tau2)
$$

In [ ]:
def generate_data(
    N: int,
    pi: float,
    sigma2: float = 1.0,
    tau2: float = 1.0,
    rng: np.random.Generator | None = None,
):

    if rng is None:
        rng = np.random.default_rng()
    if not (0.0 < pi < 1.0):
        raise ValueError("pi must be in (0,1).")
    if sigma2 < 0 or tau2 < 0:
        raise ValueError("sigma2 and tau2 must be nonnegative.")

    gamma = rng.binomial(1, pi, size=N).astype(int)
    mu = rng.normal(0.0, np.sqrt(sigma2), size=N) * gamma
    z = mu + rng.normal(0.0, np.sqrt(tau2), size=N)
    return mu, z, gamma



nonzeros = 16 / 200, z mean = -0.028
